In [34]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, AveragePooling2D, Conv2D, Flatten
from keras.layers import Lambda
from keras import backend as K # tensorflow

import coremltools
from coremltools.proto import NeuralNetwork_pb2

import os

In [35]:
kears_file = "./KerasMNIST_customlayer.h5"
coreml_file = './KerasMNIST_customlayer.mlmodel'

In [36]:
def build_and_learn_keras_model():

    # Just sigmoid
    def custom_sigmoid_activation(x):
        return K.sigmoid(x)

    # Just relu
    def custom_relu_activation(x):
        return K.relu(x)

    batch_size = 128
    num_classes = 10
    epochs = 10

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print(x_train.shape)
    print(x_train.shape[1:])

    img_rows = 28
    img_cols = 28

    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # 入力値の正規化
    x_train /= 255
    x_test /= 255

    # 教師データをクラス分類のデータに変換
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # ネットワーク設計
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', input_shape=input_shape))

    # custom layer
    model.add(Lambda(custom_sigmoid_activation))

    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(128))

    # custom layer
    model.add(Lambda(custom_relu_activation))

    model.add(Dense(num_classes, activation='softmax'))

    # ネットワークの構成を出力する
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [37]:
if not os.path.exists(kears_file):
    keras_model = build_and_learn_keras_model()
    keras_model.save(kears_file)

(60000, 28, 28)
(28, 28)
(60000, 'train samples')
(10000, 'test samples')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
lambda_7 (Lambda)            (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 16)        4624      
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)              0         
_________________________________________________________________
de

In [38]:
def convert_custom_lambda_layer(layer):
    
    params = NeuralNetwork_pb2.CustomLayerParams()
    
    if layer.function.__name__ == custom_relu_activation.__name__:
        params.className = "custom_relu_activation"
        params.description = "RELU"
        return params
    elif layer.function.__name__ == custom_sigmoid_activation.__name__:
        params.className = "custom_sigmoid_activation"
        params.description = "sigmoid"
        return params
    else:
        return None

coreml_model = coremltools.converters.keras.convert(
    kears_file,
    input_names='image',
    output_names='digit',
    add_custom_layers=True,
    custom_conversion_functions={ "Lambda": convert_custom_lambda_layer }
)

coreml_model.author = u'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = u'Custom layerのサンプル'

coreml_model.input_description['image'] = u'入力画像'
coreml_model.output_description['digit'] = u'推定した数字の確率'

coreml_model.save(coreml_file)

0 : conv2d_10_input, <keras.engine.topology.InputLayer object at 0x17b027950>
1 : conv2d_10, <keras.layers.convolutional.Conv2D object at 0x17b027990>
2 : lambda_7, <keras.layers.core.Lambda object at 0x17b027b50>
3 : conv2d_11, <keras.layers.convolutional.Conv2D object at 0x17b022c10>
4 : conv2d_11__activation__, <keras.layers.core.Activation object at 0x16cf9f610>
5 : max_pooling2d_4, <keras.layers.pooling.MaxPooling2D object at 0x16ce9b510>
6 : conv2d_12, <keras.layers.convolutional.Conv2D object at 0x16ceb03d0>
7 : conv2d_12__activation__, <keras.layers.core.Activation object at 0x16cf9f0d0>
8 : flatten_4, <keras.layers.core.Flatten object at 0x16cec4dd0>
9 : dense_7, <keras.layers.core.Dense object at 0x16ceda3d0>
10 : lambda_8, <keras.layers.core.Lambda object at 0x17b027b90>
11 : dense_8, <keras.layers.core.Dense object at 0x16cf0ab90>
12 : dense_8__activation__, <keras.layers.core.Activation object at 0x16e2db710>
